In [2]:
import mysql.connector
from datetime import datetime
import mysql.connector
import pandas as pd
import warnings
from tqdm.notebook import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import time
import numpy as np 
import matplotlib.gridspec as gridspec
from matplotlib.widgets import Button
import matplotlib.dates as mdates
import gymnasium as gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from gym import spaces


In [3]:
class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)

    # 최적화 중에 다음 행동을 결정하기 위해서 하나의 요소 또는 배치를 이용해 호촐됩니다.
    # ([[left0exp,right0exp]...]) 를 반환합니다.
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [12]:

class TradingEnv(gym.Env):
    def __init__(self, window_size=200):
        super(TradingEnv, self).__init__()
        
        
        # today = "A" + datetime.now().strftime("%Y%m%d") 
        self.host = '127.0.0.1'
        self.user = 'root'
        self.databaseA = 'a20240112'
        self.password = '93150lbm!!'
        
        
        self.window_size = window_size
        self.current_step = window_size
        
        self.action_space = spaces.Discrete(2) 

        self.tables = [code for (code,) in self.get_table_names(self.host,self.user,self.password,self.databaseA)]
        
        sample_code = self.tables[1]
        
        self.df = self.table_to_dataframe(self.host,self.user,self.password,self.databaseA,sample_code)
        self.process_df = self.preprocessing(self.df)
        
        
        print(sample_code)
        print(self.process_df)
        
        
        
        

        
    def step(self,action):
        self.reward = 0 
        self.done = False
        current_price = 'Fluctuation_15s'
        
        
        fluctuation = self.process_df['Fluctuation_15s'][self.current_step]
        
        if action == 0 : # buy 
            if fluctuation > 0 : 
                self.reward = fluctuation
            elif fluctuation <= 0 : 
                self.reward = - fluctuation
        elif action == 1 : # sell
            if fluctuation < 0 : 
                self.reward = - fluctuation
            elif fluctuation > 0 : 
                self.reward = fluctuation
                
        
        
        
        if self.current_step >= len(self.process_df)-1:
            self.done = True 
        
        else : 
            self.current_step += 1
        
        return self._next_observation(), self.reward, self.done, {}
        
        
    def reset(self):
        self.current_step = self.window_size
        self.current_step += 1
        return self._next_observation()
        
    def _next_observation(self):
        # end = min(len(self.process_df), self.current_step)  # Ensure end does not exceed DataFrame length
        # start = max(0, end - self.window_size)
        start = self.current_step - self.window_size
        end = self.current_step
        obs = self.process_df.loc[start:end-1,'signal'].values
        obs = np.array(obs)
        # print(f"Start: {start}, End: {end}, Obs length: {len(obs)}")
        return obs
        
    
    
    
    def preprocessing(self,df):
        df['체결시간'] = pd.to_datetime(df['체결시간'])
        df['현재가'] = pd.to_numeric(df['현재가'].str.replace('+','').str.replace('-',''), errors='coerce')
        df['거래량'] = pd.to_numeric(df['거래량'], errors='coerce')
        df['거래대금'] = df['현재가'] * df['거래량']
        df = df.sort_values(by='체결시간', ignore_index=True)
        df['time_diff_seconds'] = df['체결시간'].diff().dt.total_seconds()
        df['exp_minus'] = np.exp(-df['time_diff_seconds'])
        df['signal'] = df['exp_minus'] * df['거래대금']

        
        df_use = df[['체결시간','거래대금', 'time_diff_seconds', '현재가','signal']]
        
        
        df_use['Fluctuation_15s'] = None

        
        intervals = 15  # 15 seconds, 30 seconds, 1 minute
        seconds = intervals 
        column_name = 'Fluctuation_15s'
        for index, row in tqdm(df_use.iterrows(),total= len(df_use)):
            base_time = row['체결시간']
            comparison_index = self.find_closest_index(df_use, base_time, seconds)
            fluctuation_rate = self.calculate_fluctuation_rate(df_use, index, comparison_index)
            df_use.at[index, column_name] = fluctuation_rate
        
        df_use = df_use.dropna()
        
        filtered_df = df_use[df_use['체결시간'].dt.time < pd.to_datetime('15:19').time()]

        filtered_df = filtered_df.dropna()
        # df_use['Fluctuation_15s'] = None
        # df_use['Fluctuation_30s'] = None
        # df_use['Fluctuation_60s'] = None
        
        # intervals = [15, 30, 60]  # 15 seconds, 30 seconds, 1 minute
        # for index, row in tqdm(df_use.iterrows(),total= len(df_use)):
        #     base_time = row['체결시간']
        #     for seconds, column_name in zip(intervals, ['Fluctuation_15s', 'Fluctuation_30s', 'Fluctuation_60s']):
        #         comparison_index = self.find_closest_index(df_use, base_time, seconds)
        #         fluctuation_rate = self.calculate_fluctuation_rate(df_use, index, comparison_index)
        #         df_use.at[index, column_name] = fluctuation_rate
        
        # df_use = df_use.dropna()
        
        
        return filtered_df 
        
        
        
        
    
    def find_closest_index(self,df, base_time, delta_seconds):
        target_time = base_time + pd.Timedelta(seconds=delta_seconds)
        closest_index = df['체결시간'].searchsorted(target_time, side='left')
        if closest_index < len(df):
            return closest_index
        return None  # Indicates no suitable index was found

    # Calculate fluctuation rate
    def calculate_fluctuation_rate(self,df, base_index, comparison_index):
        if comparison_index is None or comparison_index >= len(df):
            return np.nan  # No data available for comparison
        initial_price = df.iloc[base_index]['현재가']
        new_price = df.iloc[comparison_index]['현재가']
        return ((new_price - initial_price) / initial_price) * 100
    
    
    
    
    
    def table_to_dataframe(self,host, user, password, database, table_name):
        conn = None
        try:
            # Establish a database connection
            conn = mysql.connector.connect(
                host=host,
                user=user,
                password=password,
                database=database
            )

            # Use backticks around the table name in the query
            query = f"SELECT * FROM `{table_name}`"

            # Use pandas to read the query into a DataFrame
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                df = pd.read_sql(query, conn)

            return df

        except mysql.connector.Error as error:
            print(f"Error: {error}")
            return None
        finally:
            if conn.is_connected():
                conn.close()
        

    def get_table_names(self, host, user, password, database):
        conn = None
        cursor = None
        tables = []  # Initialize tables as an empty list

        try:
            # Establish a database connection
            conn = mysql.connector.connect(
                host=host,
                user=user,
                password=password,
                database=database
            )

            # Create a cursor object
            cursor = conn.cursor()

            # Execute a query to retrieve table names
            query = f"SELECT table_name FROM INFORMATION_SCHEMA.TABLES WHERE table_schema = '{database}'"
            cursor.execute(query)

            # Fetch all the rows
            tables = cursor.fetchall()

        except mysql.connector.Error as error:
            print(f"Error: {error}")

        finally:
            # Ensure to close resources properly
            if cursor is not None:
                cursor.close()
            if conn is not None and conn.is_connected():
                conn.close()

        return tables


In [13]:
env = TradingEnv()

C:\Users\pari0\AppData\Local\Temp\ipykernel_29456\1745847226.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_use['Fluctuation_15s'] = None


  0%|          | 0/22920 [00:00<?, ?it/s]

005290
                            체결시간     거래대금  time_diff_seconds    현재가  \
1     2024-01-12 09:41:32.930311  1519000           0.207557  43400   
2     2024-01-12 09:41:33.136453 -2687700           0.206142  43350   
3     2024-01-12 09:41:33.147454  -606900           0.011001  43350   
4     2024-01-12 09:41:33.156434  -606900           0.008980  43350   
5     2024-01-12 09:41:33.167509 -1907400           0.011075  43350   
...                          ...      ...                ...    ...   
22830 2024-01-12 15:18:53.896563  -332800           3.965066  41600   
22831 2024-01-12 15:18:56.802730   417000           2.906167  41700   
22832 2024-01-12 15:18:58.718282    83400           1.915552  41700   
22833 2024-01-12 15:18:58.997987   -41650           0.279705  41650   
22834 2024-01-12 15:18:59.143621  2040850           0.145634  41650   

             signal Fluctuation_15s  
1      1.234289e+06        0.345622  
2     -2.187029e+06        0.461361  
3     -6.002601e+05       

In [16]:
import torch 

n_observations = 200
n_actions = 2

model = DQN(n_observations,n_actions)
model.load_state_dict(torch.load('policy_net.pth'))
model.eval() 

DQN(
  (layer1): Linear(in_features=200, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=128, bias=True)
  (layer3): Linear(in_features=128, out_features=2, bias=True)
)

In [18]:
total_cnt = 0 
true_cnt = 0 

for i in tqdm(range(len(env.process_df))):

    try: 
        signal_data = env.process_df['signal'][i:i+200].values
        fluc = env.process_df['Fluctuation_15s'][i:i+200].values[-1]


        input_data = torch.tensor(signal_data,dtype=torch.float32)
        action = model(input_data)
        # print(fluc)
        # print(action.argmax())
        if fluc > 0 and action.argmax() > 0 :
            true_cnt += 1
        total_cnt +=1 
        
    except : 
        break
    
print(f'{true_cnt/total_cnt*100:.2f}%')

  0%|          | 0/22834 [00:00<?, ?it/s]

22.14%
